In [1]:
import pandas as pd
from pathlib import Path

from scripts.utils_etl import (
    clean_sales_train,
    aggregate_to_monthly,
    fill_sparse_timeseries,
    enrich_items,
    enrich_categories,
    enrich_shops,
    join_with_references,
    create_basic_features,
    prepare_test_data,
    save_processed_data,
    validate_etl_results,
)

import warnings

warnings.filterwarnings("ignore")

In [2]:
raw_data_path = Path("../data/raw/")
processed_data_path = Path("../data/processed/")
processed_data_path.mkdir(parents=True, exist_ok=True)

sales_train = pd.read_csv(raw_data_path / "sales_train.csv")
test = pd.read_csv(raw_data_path / "test.csv")
items = pd.read_csv(raw_data_path / "items.csv")
shops = pd.read_csv(raw_data_path / "shops.csv")
item_categories = pd.read_csv(raw_data_path / "item_categories.csv")

In [3]:
sales_train["date"] = pd.to_datetime(sales_train["date"], format="%d.%m.%Y")

In [4]:
sales_train_clean = clean_sales_train(sales_train)

print(f"После очистки: {len(sales_train_clean):,} записей")
print(f"Удалено записей: {len(sales_train) - len(sales_train_clean):,}")

# Проверка новых столбцов
print("\nНовые столбцы:")
print(f"is_return: {sales_train_clean['is_return'].sum():,} возвратов")
print(
    f"item_cnt_day_abs: диапазон {sales_train_clean['item_cnt_day_abs'].min():.0f} - {sales_train_clean['item_cnt_day_abs'].max():.0f}"
)

print("\nЭтап 2 завершен.")

После очистки: 2,935,845 записей
Удалено записей: 4

Новые столбцы:
is_return: 7,356 возвратов
item_cnt_day_abs: диапазон 1 - 669

Этап 2 завершен.


In [5]:
print("Этап 3: Агрегация до месячного уровня")
print(f"Исходное количество записей: {len(sales_train_clean):,}")

# Применяем агрегацию
sales_monthly = aggregate_to_monthly(sales_train_clean)

print(f"После агрегации: {len(sales_monthly):,} записей")
print(f"Размерность: {sales_monthly.shape}")

# Проверка структуры
print(f"\nСтолбцы: {list(sales_monthly.columns)}")
print("\nПервые 5 строк:")
print(sales_monthly.head())

# Статистика по основным метрикам
print("\nСтатистика по item_cnt_month:")
print(sales_monthly["item_cnt_month"].describe())

print("\nЭтап 3 завершен.")

Этап 3: Агрегация до месячного уровня
Исходное количество записей: 2,935,845
После агрегации: 1,609,122 записей
Размерность: (1609122, 11)

Столбцы: ['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'item_price_mean', 'item_price_median', 'item_price_min', 'item_price_max', 'transactions_count', 'returns_count', 'returns_sum']

Первые 5 строк:
   shop_id  item_id  date_block_num  item_cnt_month  item_price_mean  \
0        0       30               1            31.0            265.0   
1        0       31               1            11.0            434.0   
2        0       32               0             6.0            221.0   
3        0       32               1            10.0            221.0   
4        0       33               0             3.0            347.0   

   item_price_median  item_price_min  item_price_max  transactions_count  \
0              265.0           265.0           265.0                   9   
1              434.0           434.0           434.0        

In [6]:
# Этап 4: Обработка разреженных временных рядов

print("Этап 4: Обработка разреженных временных рядов")
print(f"Исходное количество записей: {len(sales_monthly):,}")

# Подсчет активных пар
active_pairs = sales_monthly[["shop_id", "item_id"]].drop_duplicates()
print(f"Активных пар (shop_id x item_id): {len(active_pairs):,}")

# Применяем заполнение
sales_monthly_filled = fill_sparse_timeseries(sales_monthly)

print(f"После заполнения: {len(sales_monthly_filled):,} записей")
print(f"Ожидаемое количество: {len(active_pairs) * 34:,} (пары x 34 месяца)")

# Проверка заполнения
print("\nПроверка заполнения:")
print(
    f"  Записей с item_cnt_month = 0: {(sales_monthly_filled['item_cnt_month'] == 0).sum():,}"
)
print(
    f"  Записей с item_cnt_month > 0: {(sales_monthly_filled['item_cnt_month'] > 0).sum():,}"
)

# Проверка пропусков
print("\nПроверка пропусков:")
print(sales_monthly_filled.isnull().sum())

# Пример: проверка одной пары
example_pair = sales_monthly_filled[
    (sales_monthly_filled["shop_id"] == 0) & (sales_monthly_filled["item_id"] == 30)
].head(10)
print("\nПример: shop_id=0, item_id=30 (первые 10 месяцев):")
print(example_pair[["date_block_num", "item_cnt_month", "item_price_median"]])

print("\nЭтап 4 завершен.")

Этап 4: Обработка разреженных временных рядов
Исходное количество записей: 1,609,122
Активных пар (shop_id x item_id): 424,123
После заполнения: 14,420,182 записей
Ожидаемое количество: 14,420,182 (пары x 34 месяца)

Проверка заполнения:
  Записей с item_cnt_month = 0: 12,811,060
  Записей с item_cnt_month > 0: 1,609,122

Проверка пропусков:
shop_id               0
item_id               0
date_block_num        0
item_cnt_month        0
item_price_mean       0
item_price_median     0
item_price_min        0
item_price_max        0
transactions_count    0
returns_count         0
returns_sum           0
dtype: int64

Пример: shop_id=0, item_id=30 (первые 10 месяцев):
   date_block_num  item_cnt_month  item_price_median
0               0             0.0              265.0
1               1            31.0              265.0
2               2             0.0              265.0
3               3             0.0              265.0
4               4             0.0              265.0
5        

In [7]:
print("Этап 5: Обогащение справочниками")

# Обогащаем items
print("\n1. Обогащение items...")
items_enriched = enrich_items(items, item_categories, sales_train_clean)
print(
    f"Добавлены колонки: {[col for col in items_enriched.columns if col not in items.columns]}"
)
print("Пример:")
print(
    items_enriched[
        ["item_id", "item_name", "main_category", "sub_category", "has_sales_history"]
    ].head()
)

# Обогащаем shops
print("\n2. Обогащение shops...")
shops_enriched = enrich_shops(shops)
print(
    f"Добавлены колонки: {[col for col in shops_enriched.columns if col not in shops.columns]}"
)
print("Пример:")
print(
    shops_enriched[
        ["shop_id", "shop_name", "city", "shop_type", "is_online", "is_franchise"]
    ].head()
)

# Обогащаем item_categories
print("\n3. Обогащение item_categories...")
item_categories_enriched = enrich_categories(item_categories)
print(
    f"Добавлены колонки: {[col for col in item_categories_enriched.columns if col not in item_categories.columns]}"
)
print("Пример:")
print(item_categories_enriched.head())

print("\nЭтап 5 завершен.")

Этап 5: Обогащение справочниками

1. Обогащение items...
Добавлены колонки: ['main_category', 'sub_category', 'is_digital', 'has_sales_history']
Пример:
   item_id                                          item_name main_category  \
0        0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D          Кино   
1        1  !ABBYY FineReader 12 Professional Edition Full...     Программы   
2        2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D          Кино   
3        3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D          Кино   
4        4        ***КОРОБКА (СТЕКЛО)                       D          Кино   

               sub_category  has_sales_history  
0                       DVD               True  
1  Для дома и офиса (Цифра)               True  
2                       DVD               True  
3                       DVD               True  
4                       DVD               True  

2. Обогащение shops...
Добавлены колонки: ['city', 'shop_type', 'is_online', 

In [8]:
print("Этап 6: Джойн данных")
print(f"Исходное количество записей: {len(sales_monthly_filled):,}")

# Применяем джойн
sales_monthly_enriched = join_with_references(
    sales_monthly_filled, items_enriched, shops_enriched
)

print(f"После джойна: {len(sales_monthly_enriched):,} записей")
print(f"Размерность: {sales_monthly_enriched.shape}")

# Проверка структуры
print(f"\nСтолбцы ({len(sales_monthly_enriched.columns)}):")
print(list(sales_monthly_enriched.columns))

# Проверка пропусков
print("\nПроверка пропусков:")
missing = sales_monthly_enriched.isnull().sum()
print(missing[missing > 0])

# Пример данных
print("\nПример данных (первые 3 строки):")
print(sales_monthly_enriched.head(3))

print("\nЭтап 6 завершен.")

Этап 6: Джойн данных
Исходное количество записей: 14,420,182
После джойна: 14,420,182 записей
Размерность: (14420182, 22)

Столбцы (22):
['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'item_price_mean', 'item_price_median', 'item_price_min', 'item_price_max', 'transactions_count', 'returns_count', 'returns_sum', 'item_name', 'item_category_id', 'main_category', 'sub_category', 'is_digital', 'has_sales_history', 'shop_name', 'city', 'shop_type', 'is_online', 'is_franchise']

Проверка пропусков:
Series([], dtype: int64)

Пример данных (первые 3 строки):
   shop_id  item_id  date_block_num  item_cnt_month  item_price_mean  \
0        0       30               0             0.0            265.0   
1        0       30               1            31.0            265.0   
2        0       30               2             0.0            265.0   

   item_price_median  item_price_min  item_price_max  transactions_count  \
0              265.0           265.0           265.0             

In [9]:
print("Этап 7: Создание базовых фичей")
print(f"Исходное количество записей: {len(sales_monthly_enriched):,}")

sales_monthly_with_features = create_basic_features(sales_monthly_enriched)

print(f"После создания фичей: {len(sales_monthly_with_features):,} записей")
print(f"Размерность: {sales_monthly_with_features.shape}")

# Проверка созданных фичей
print("\nСозданные фичи:")
new_features = [
    "month",
    "year",
    "is_holiday_season",
    "lag_1",
    "lag_2",
    "lag_3",
    "lag_6",
    "lag_12",
    "mean_3m",
    "mean_6m",
    "mean_12m",
    "item_total_sales",
    "item_avg_price",
    "shop_total_sales",
]
print(
    f"  Временные: {[f for f in new_features if f in ['month', 'year', 'is_holiday_season']]}"
)
print(f"  Лаги: {[f for f in new_features if f.startswith('lag_')]}")
print(f"  Скользящие средние: {[f for f in new_features if f.startswith('mean_')]}")
print(f"  Агрегаты по товару: {[f for f in new_features if f.startswith('item_')]}")
print(f"  Агрегаты по магазину: {[f for f in new_features if f.startswith('shop_')]}")

# Проверка пропусков в новых фичах
print("\nПроверка пропусков в фичах:")
missing = sales_monthly_with_features[new_features].isnull().sum()
print(missing[missing > 0])

# Пример данных
print("\nПример данных (shop_id=0, item_id=30, первые 5 месяцев):")
example = sales_monthly_with_features[
    (sales_monthly_with_features["shop_id"] == 0)
    & (sales_monthly_with_features["item_id"] == 30)
].head(5)
print(
    example[
        [
            "date_block_num",
            "month",
            "year",
            "item_cnt_month",
            "lag_1",
            "lag_2",
            "mean_3m",
        ]
    ]
)

print("\nЭтап 7 завершен.")

Этап 7: Создание базовых фичей
Исходное количество записей: 14,420,182
После создания фичей: 14,420,182 записей
Размерность: (14420182, 36)

Созданные фичи:
  Временные: ['month', 'year', 'is_holiday_season']
  Лаги: ['lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12']
  Скользящие средние: ['mean_3m', 'mean_6m', 'mean_12m']
  Агрегаты по товару: ['item_total_sales', 'item_avg_price']
  Агрегаты по магазину: ['shop_total_sales']

Проверка пропусков в фичах:
Series([], dtype: int64)

Пример данных (shop_id=0, item_id=30, первые 5 месяцев):
   date_block_num  month  year  item_cnt_month  lag_1  lag_2    mean_3m
0               0      1  2013             0.0    0.0    0.0   0.000000
1               1      2  2013            31.0    0.0    0.0  15.500000
2               2      3  2013             0.0   31.0    0.0  10.333333
3               3      4  2013             0.0    0.0   31.0  10.333333
4               4      5  2013             0.0    0.0    0.0   0.000000

Этап 7 завершен.


In [10]:
print("Этап 8: Подготовка тестовых данных")
print(f"Исходное количество записей в test: {len(test):,}")

# Применяем подготовку тестовых данных
test_enriched = prepare_test_data(
    test, sales_monthly_with_features, items_enriched, shops_enriched
)

print(f"После подготовки: {len(test_enriched):,} записей")
print(f"Размерность: {test_enriched.shape}")

# Проверка структуры
print(f"\nСтолбцы ({len(test_enriched.columns)}):")
print(list(test_enriched.columns))

# Проверка флагов истории
print("\nПроверка флагов истории:")
print(f"  Товаров с историей: {test_enriched['has_sales_history'].sum():,}")
print(f"  Новых товаров: {(~test_enriched['has_sales_history']).sum():,}")
print(f"  Пар с историей: {test_enriched['has_pair_history'].sum():,}")
print(f"  Новых пар: {(~test_enriched['has_pair_history']).sum():,}")

# Проверка пропусков
print("\nПроверка пропусков:")
missing = test_enriched.isnull().sum()
print(missing[missing > 0])

# Пример данных
print("\nПример данных (первые 3 строки):")
print(test_enriched.head(3))

# Пример новых пар
print("\nПример новых пар (без истории):")
new_pairs = test_enriched[~test_enriched["has_pair_history"]].head(3)
print(
    new_pairs[
        [
            "ID",
            "shop_id",
            "item_id",
            "has_sales_history",
            "has_pair_history",
            "main_category",
            "city",
            "shop_type",
        ]
    ]
)

print("\nЭтап 8 завершен.")

Этап 8: Подготовка тестовых данных
Исходное количество записей в test: 214,200
После подготовки: 214,200 записей
Размерность: (214200, 34)

Столбцы (34):
['ID', 'shop_id', 'item_id', 'date_block_num', 'has_pair_history', 'item_name', 'item_category_id', 'main_category', 'sub_category', 'is_digital', 'has_sales_history', 'shop_name', 'city', 'shop_type', 'is_online', 'is_franchise', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12', 'mean_3m', 'mean_6m', 'mean_12m', 'month', 'year', 'is_holiday_season', 'item_total_sales', 'item_avg_price', 'shop_total_sales', 'item_avg_sales_all_time', 'item_avg_price_all_time', 'shop_avg_sales_all_time', 'category_avg_sales_in_shop']

Проверка флагов истории:
  Товаров с историей: 198,954
  Новых товаров: 15,246
  Пар с историей: 111,404
  Новых пар: 102,796

Проверка пропусков:
Series([], dtype: int64)

Пример данных (первые 3 строки):
   ID  shop_id  item_id  date_block_num  has_pair_history  \
0   0        5     5037              34              True   

In [11]:
print("Этап 9: Валидация и отчетность")

validation_results = validate_etl_results(
    sales_train_original=sales_train,
    sales_train_clean=sales_train_clean,
    sales_monthly=sales_monthly,
    sales_monthly_with_features=sales_monthly_with_features,
    test_enriched=test_enriched,
    items_enriched=items_enriched,
    shops_enriched=shops_enriched,
    item_categories_enriched=item_categories_enriched,
)

print("\nЭтап 9 завершен.")

Этап 9: Валидация и отчетность

1. СТАТИСТИКА ПО ОЧИСТКЕ ДАННЫХ
Удалено выбросов: 4 записей
Возвратов (отрицательных значений): 7,356

2. ПРОВЕРКА РАЗМЕРНОСТЕЙ
sales_monthly: 1,609,122 записей
sales_monthly_with_features: 14,420,182 записей (424,123 пар x 34 месяца)
test_enriched: 214,200 записей
items_enriched: 22,170 записей
shops_enriched: 60 записей
item_categories_enriched: 84 записей

3. ПРОВЕРКА ПРОПУСКОВ
Нет пропусков в ключевых столбцах train
Нет пропусков в ключевых столбцах test
Нет пропусков в фичах train
Нет пропусков в фичах test

4. ПРОВЕРКА КОРРЕКТНОСТИ АГРЕГАЦИИ
Агрегация item_total_sales корректна

5. ПРОВЕРКА КОРРЕКТНОСТИ ДЖОЙНОВ
Все товары из sales присутствуют в items_enriched
Все магазины из sales присутствуют в shops_enriched

6. ПРОВЕРКА КОРРЕКТНОСТИ ФИЧЕЙ
lag_1: все значения >= 0
lag_2: все значения >= 0
lag_3: все значения >= 0
lag_6: все значения >= 0
lag_12: все значения >= 0
mean_3m: все значения >= 0
mean_6m: все значения >= 0
mean_12m: все значения >= 0
m

In [12]:
print("Этап 10: Сохранение обработанных данных")

save_processed_data(
    sales_monthly=sales_monthly,
    sales_monthly_with_features=sales_monthly_with_features,
    test_enriched=test_enriched,
    items_enriched=items_enriched,
    shops_enriched=shops_enriched,
    item_categories_enriched=item_categories_enriched,
    output_path=processed_data_path,
)

print("\nЭтап 10 завершен.")

Этап 10: Сохранение обработанных данных
Сохранение обработанных данных в формате Parquet...
sales_monthly_clean.parquet: 1,609,122 строк, 20.11 MB
sales_monthly_with_features.parquet: 14,420,182 строк, 351.55 MB
test_enriched.parquet: 214,200 строк, 15.09 MB
items_enriched.parquet: 22,170 строк, 0.68 MB
shops_enriched.parquet: 60 строк, 0.00 MB
item_categories_enriched.parquet: 84 строк, 0.01 MB

Все данные сохранены в ../data/processed

Этап 10 завершен.
